<a href="https://colab.research.google.com/github/yuhui-0611/ESAA/blob/main/ESAA_YB_WEEK07_2_Code_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 주제 및 데이터

Loan Approval Prediction — 대출 승인 예측
> 은행 고객의 개인 정보와 재정 정보를 바탕으로, 대출 신청이 승인(Y)될지 거절(N)될지를 예측하는 이진 분류(Binary Classification) 문제

| 변수명                   | 설명                             | 변수 유형          |
| --------------------- | ------------------------------ | -------------- |
| **Loan_ID**           | 신청자 식별용 ID                     | 식별자            |
| **Gender**            | 성별 (Male/Female)               | 범주형            |
| **Married**           | 결혼 여부 (Yes/No)                 | 범주형            |
| **Dependents**        | 부양가족 수                         | 범주형            |
| **Education**         | 교육 수준 (Graduate/Not Graduate)  | 범주형            |
| **Self_Employed**     | 자영업 여부                         | 범주형            |
| **ApplicantIncome**   | 신청자 월 소득                       | 수치형            |
| **CoapplicantIncome** | 공동 신청자 월 소득                    | 수치형            |
| **LoanAmount**        | 대출 금액                          | 수치형            |
| **Loan_Amount_Term**  | 대출 상환 기간 (개월)                  | 수치형            |
| **Credit_History**    | 신용 기록 (1: 있음, 0: 없음)           | 수치형            |
| **Property_Area**     | 주택 위치 (Urban/Semiurban/Rural)  | 범주형            |
| **Loan_Status**       | 대출 승인 여부 (Y/N) → **Target 변수** | 범주형 (이진 분류 타깃) |


# 코드

**< Feature Engineering >**

**A. Filtered dataset**

- 목적: 불필요하거나 분석에 방해되는 데이터를 제거하고, 학습용으로 정제된 데이터셋을 만듦

- 사용된 개념:

    > drop()으로 식별자 컬럼(ID 등)을 제거함.

    > dropna() 또는 조건 필터링을 통해 결측치와 이상치를 제거함

    > 일부 수치형 변수에 대해서는 극단값을 잘라내어 모델이 치우치지 않도록 함(IQR clipping)

- 의의: 이후 Feature 생성 단계에서 잡음(noise)을 최소화해 모델의 안정성을 높임

**B. Generate features**

- 목적: 기존 변수들로부터 추가적인 설명력을 가지는 파생변수를 생성

- 사용된 기법:

    > 조합형 변수: 예를 들어 loan_to_income = loan_amnt / income 형태의 비율 변수 생성 → 이는 대출 규모 대비 소득 수준을 반영하여 채무 위험도를 나타냄

    > 로그 변환(log1p): LoanAmount처럼 한쪽으로 치우친 분포를 정규화하기 위함

    > 범주 병합: 희소범주(빈도 낮은 값)를 ‘Other’ 그룹으로 통합함. 이는 모델이 작은 표본에 과적합되는 것을 방지함

    > 나이 구간화(bin): 연속형 변수(person_age)를 구간화하여 비선형 관계를 더 잘 포착함

- 효과: 모델이 단순히 원시 데이터에 의존하지 않고, 의미 있는 지표(예: 부채상환비율, 금리×기간 등)를 학습하게 함

**C. Target data**

- 목적: 목표 변수(대출 승인 여부)를 학습 가능한 형태로 변환

- 사용된 처리:

    > Loan_Status를 ‘Y’→1, ‘N’→0으로 변환

    > 클래스 불균형이 존재할 경우, class_weight='balanced' 옵션을 적용하거나 오버샘플링(SMOTE)을 고려

- 효과: 학습 시 모델이 승인/거절 클래스 모두를 균형 있게 학습하게 함

**D. Transformed feature and test**

- 목적: 학습용 데이터와 테스트용 데이터를 동일한 전처리 로직으로 변환

- 사용된 방식:

    > 범주형 변수에 One-Hot Encoding(get_dummies)을 적용하여 수치형으로 변환

    > 연속형 변수에는 표준화(StandardScaler) 또는 정규화(MinMaxScaler)를 적용

    > 훈련/테스트 세트의 컬럼 구조를 일치시킴(모델 입력 불일치를 방지함)

- 효과: 모델 입력 간 일관성을 유지하여 예측 시 오류를 방지함

**< Training Arc >**

**1. Random Forest**

- 사용 이유:

    > 비선형성과 변수 상호작용을 잘 포착하며, 결측치 처리에도 비교적 강건

    > 변수 중요도를 파악하기 쉬워 해석력도 확보 가능

- 설정 요소:

    > 주요 하이퍼파라미터는 n_estimators, max_depth, min_samples_split 등

    > 과적합 방지를 위해 트리 깊이와 분기 최소 샘플 수를 조정

- 역할: baseline 모델로 사용되어, 이후 LightGBM/ CatBoost와의 성능 비교 기준이 됨

**2. LightGBM**

- 사용 이유:

    > 빠른 학습 속도와 높은 예측 성능을 보유함.

    > 대용량 데이터 및 결측치 자동 처리에 유리함.

- 설정 및 특징:

    > Gradient Boosting 계열 모델로, num_leaves, learning_rate, max_depth 등 주요 하이퍼파라미터를 조정함.

    > categorical_feature 인자를 통해 범주형 데이터를 직접 처리할 수 있음.

- 특징적 요소:

    > 기존 RandomForest 대비 더 미세한 가중치 조정을 통해 오차를 줄임.

    > Early stopping 기능을 활용해 과적합을 방지함.

- 효과: 가장 높은 ROC-AUC를 보였으며, 최종 예측 모델로 채택됨

**3. CatBoost**

- 사용 이유:

    > 범주형 데이터를 자동 인코딩하며, Label Encoding보다 안정적인 target-based 인코딩 방식을 사용

    > 학습 중 overfitting detector 기능이 내장되어 있음

- 설정 및 특징:

    > 주요 파라미터: iterations, depth, learning_rate, l2_leaf_reg

    > GPU 학습을 지원해 속도 향상을 달성

- 효과: LightGBM과 유사한 성능을 보였지만, 범주형 처리에서 더 안정적인 결과를 냄

**< Predict Arc >**

학습된 각 모델로 테스트 데이터를 예측

predict_proba() 결과 중 0.5 기준으로 승인 여부를 분류

모델별 결과를 비교하고, 최종적으로 가장 높은 정확도·AUC를 보이는 모델을 선택

# 차별점 및 배운점

- 훈련(train)–검증(validation)–테스트(test) 분할의 의미 → 표본 분할은 통계적 추정의 신뢰도를 평가하는 핵심 절차

- 교차검증(Cross Validation) 을 통한 분산-편향 트레이드오프 확인 → 모델이 지나치게 복잡하면 분산(variance)이 커지고 일반화가 떨어짐

- 정규화(Regularization) 와 Early Stopping 등의 통계적 안정화 기법 적용 → L1/L2 penalty나 학습률 조절로 분산을 줄이는 방법을 배움

- AUC는 '임의의 두 샘플 중 하나가 양성 클래스일 확률'을 의미

- Precision/Recall은 오탐(False Positive) 과 미탐(False Negative) 의 통계적 균형을 보여줌

- F1-score는 이 둘의 조화평균(harmonic mean) 으로 모델의 안정성을 판단함